In [2]:
import numpy as np
import matplotlib.pyplot as plt
import cv2, glob
from PIL import Image
import tqdm, json, os

with open('./ffhq_pair2.json', 'r') as f:
    pair = json.load(f)['pair']

inpath = '/data/mint/sampling/TPAMI/sota/TR/TREval_May/output_new/'
original_bg = '/data/mint/DPM_Dataset/ffhq_256_with_anno/ffhq_256/valid/'
mask = glob.glob(f'{inpath}/*_mask.png')
for f in tqdm.tqdm(mask):
    mask_img = Image.open(f).convert('L')
    img = Image.open(f.replace('_mask', ''))
    h, w, _ = np.array(img).shape
    # print(np.array(mask_img))
    mask_img_arr = np.array(mask_img) != 0
    # print(mask_img_arr)
    img_arr = np.array(img)
    pair_idx = f.split('/')[-1].split('_')[0]
    src_name = pair[pair_idx]['src']
    dst_name = pair[pair_idx]['dst']
    org = Image.open(original_bg + src_name)
    org_pad = np.zeros_like(img_arr)
    org_pad[128:h-128, ...] = org.resize((w, w))
    
    out_path = '/data/mint/sampling/TPAMI/sota/TR/TREval_May/output_final/'
    out_path = f'{out_path}/src={src_name}/dst={dst_name}/'
    os.makedirs(out_path, exist_ok=True)
    out = img_arr * mask_img_arr[..., None] + (org_pad * ~mask_img_arr[..., None])
    out = out.astype(np.uint8)
    
    org_pad = org_pad.astype(np.uint8)[128:h-128, ...]
    out = out.astype(np.uint8)[128:h-128, ...]
    mask_img_arr = mask_img_arr.astype(np.uint8)[128:h-128, ...]
    
    Image.fromarray(org_pad).resize((256, 256)).save(out_path + f'res_frame0.png')
    Image.fromarray(out).resize((256, 256)).save(out_path + f'res_frame1.png')
    Image.fromarray(mask_img_arr).resize((256, 256)).save(out_path + f'mask_frame1.png')
    

  0%|          | 0/3085 [00:00<?, ?it/s]

100%|██████████| 3085/3085 [07:46<00:00,  6.61it/s]
